<a href="https://colab.research.google.com/github/MSebastian3/Evaluacion/blob/main/Actividad_5_(Evaluaci%C3%B3n).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalamos librería para análisis de variables categóricas
#%pip install funpymodeling

In [30]:
#Importamos librerías requeridas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#from funpymodeling.exploratory import freq_tbl 

In [ ]:
#Cargar archivo csv desde equipo
#from google.colab import files
#files.upload()

# ***California_EUA.csv***

## **ETAPA 1**

In [31]:
dfeua = pd.read_csv('California_EUA.csv', encoding = 'unicode_escape')
dfeua.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6936 entries, 0 to 6935
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   ï»¿id                                         6936 non-null   float64
 1   listing_url                                   6936 non-null   object 
 2   scrape_id                                     6936 non-null   float64
 3   last_scraped                                  6936 non-null   object 
 4   source                                        6936 non-null   object 
 5   name                                          6936 non-null   object 
 6   description                                   6889 non-null   object 
 7   neighborhood_overview                         4850 non-null   object 
 8   picture_url                                   6936 non-null   object 
 9   host_id                                       6936 non-null   i

In [32]:
#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=dfeua.isnull().sum().sum()
valores_nulos

48877

### Filtros

In [33]:
#Crear filtro numerico y no numerico
filtroeuanumerico = dfeua.select_dtypes(include=['float64', 'int64'])
#filtroeuanumerico.info()

In [34]:
filtroeua = dfeua.select_dtypes('object')
#filtroeuanonumerico.info()

### Nulos

In [35]:
#sustituir los nulos por el promedio de la columna
data1 = filtroeuanumerico.copy()
filtroeuanumerico = data1.fillna(data1.mean())


#sustituir los nulos por 'sin registro'
filtroeua = filtroeua.fillna('sin registro')


In [36]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

20808

In [37]:
#Checar si hay valores nulos por columna    NO NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeua.isnull().sum().sum()
valores_nulos

0

In [38]:
#NUMERICO
#Se eliminaron las columnas 'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated'
#ya que son numericas pero no tienen ningun registro y no aportan valor al análisis
filtroeuanumerico = filtroeuanumerico.drop(['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated'], axis=1)

#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

0

### Outliers **MÉTODO DE IQR (RANGO INTER QUARTILICO)**

In [39]:
#Método aplicando Cuartiles. Encuentro cuartiles 0.25 y 0.75
y=filtroeuanumerico

percentile25=y.quantile(0.25) #Q1
percentile75=y.quantile(0.75) #Q3
iqr= percentile75 - percentile25

Limite_Superior_iqr= percentile75 + 1.5*iqr
Limite_Inferior_iqr= percentile25 - 1.5*iqr
#print("Limite superior permitido", Limite_Superior_iqr)
#print("Limite inferior permitido", Limite_Inferior_iqr)

In [40]:
#Encontramos Ouliers
outliers_iqr= filtroeuanumerico[(y>Limite_Superior_iqr)|(y<Limite_Inferior_iqr)]
#outliers_iqr.head()

In [41]:
#Obtenemos datos limpios
data_clean_iqr1= filtroeuanumerico[(y<=Limite_Superior_iqr)&(y>=Limite_Inferior_iqr)]
#data_clean_iqr1.head()

In [42]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data_clean_iqr1.isnull().sum().sum()
valores_nulos

17526

In [43]:
#Reemplazamos valores atípicos (nulos) del dataframe con "mean"
#Realizamos una copia del dataframe
data5_iqr=filtroeuanumerico.copy()
data5_iqr=data5_iqr.fillna(round(filtroeuanumerico.mean(),1))
#data5_iqr

In [44]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data5_iqr.isnull().sum().sum()
valores_nulos

0

### Concatenar

In [45]:
concatenadoeua = pd.concat([data5_iqr, filtroeua], axis=1)
concatenadoeua.head()

,ï»¿id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,...,room_type,bathrooms_text,amenities,price,has_availability,calendar_last_scraped,first_review,last_review,license,instant_bookable
0,958.0,2.020000e+13,1169,1,1,37.77028,-122.43317,3,1.0,2.0,...,Entire home/apt,1 bath,"[""Oven"", ""Self check-in"", ""Cooking basics"", ""L...",$202.00,t,06/03/2023,23/07/2009,19/02/2023,City Registration Pending,f
1,5858.0,2.020000e+13,8904,2,2,37.74474,-122.42089,4,2.0,2.0,...,Entire home/apt,1 bath,"[""Essentials"", ""Carbon monoxide alarm"", ""Hange...",$235.00,t,06/03/2023,03/05/2009,06/08/2017,sin registro,f
2,8142.0,2.020000e+13,21994,13,14,37.76555,-122.45213,1,1.0,1.0,...,Private room,4 shared baths,"[""Security cameras on property"", ""Host greets ...",$56.00,t,06/03/2023,08/09/2014,27/10/2022,sin registro,f
3,8339.0,2.020000e+13,24215,2,6,37.77564,-122.43642,4,2.0,2.0,...,Entire home/apt,1.5 baths,"[""Oven"", ""Cooking basics"", ""TV"", ""Smoke alarm""...",$575.00,t,06/03/2023,25/09/2009,28/06/2019,STR-0000264,f
4,8739.0,2.020000e+13,7149,2,2,37.76030,-122.42197,3,1.0,1.0,...,Private room,1 private bath,"[""Paid parking garage off premises"", ""Luggage ...",$110.00,t,06/03/2023,01/08/2009,25/02/2023,STR-0000028,f


### Guardar a CSV

In [46]:
#Convertir archivo filtrado a CSV
#concatenadoeua.to_csv("EUA_limpio.csv")

In [47]:
#descargar archivo filtrado en csv
from google.colab import files  
#files.download("EUA_limpio.csv")

## **ETAPA 2**

In [48]:
#filtroa
#se cambiaron los valores "sin registro" por "5%" para poder identificarlos ya que el dataset no tiene registros con ese valor
concatenadoeua["host_acceptance_rate"] = concatenadoeua["host_acceptance_rate"].replace("sin registro", "5%")
#se elimino el "%"
concatenadoeua['host_acceptance_rate'] = pd.to_numeric(concatenadoeua['host_acceptance_rate'].str.replace('%', ''))



In [49]:
#filtroe
#cambiar Half-bath, Shared half-bath, Private half-bath, sin registro
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.replace('Half-bath', '0.5').str.replace('Shared half-bath', '0.5').str.replace('Private half-bath', '0.5').str.replace('sin registro', '0')

#se extraen los numeros y se guardan como string en la misma columna
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.extract('(\d+\.?\d*)', expand=False)

#cambiar columna a float
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].astype(float)

In [50]:
#filtrof
#se elimino el "$" y ","
#se cambia la columna "price" a float para poder compararla
concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


<ipython-input-50-af53cb0f8d5c>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


### **Filtros**

In [51]:
filtroa_eua = concatenadoeua[concatenadoeua["host_acceptance_rate"] > 50]
filtrob_eua = concatenadoeua[concatenadoeua["host_is_superhost"] == "t"]
filtroc_eua = concatenadoeua[concatenadoeua["host_identity_verified"] == "f"]
filtrod_eua = concatenadoeua[concatenadoeua["property_type"].isin(["Private room", "Hotel room"])]
filtroe_eua = concatenadoeua[concatenadoeua["bathrooms_text"] > 1]
filtrof_eua = concatenadoeua.loc[(concatenadoeua['price'] > 10000) & (concatenadoeua['room_type'] == 'Entire home/apt')]
filtrog_eua = concatenadoeua[concatenadoeua["review_scores_cleanliness"] > 4.5]
filtroh_eua = concatenadoeua[concatenadoeua["review_scores_value"] > 4.9]
filtroi_eua = concatenadoeua[concatenadoeua["availability_365"] < 100]
filtroj_eua = concatenadoeua[concatenadoeua["host_response_time"] == "within an hour"]

In [52]:
concatenadoeua.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6936 entries, 0 to 6935
Data columns (total 72 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   ï»¿id                                         6936 non-null   float64
 1   scrape_id                                     6936 non-null   float64
 2   host_id                                       6936 non-null   int64  
 3   host_listings_count                           6936 non-null   int64  
 4   host_total_listings_count                     6936 non-null   int64  
 5   latitude                                      6936 non-null   float64
 6   longitude                                     6936 non-null   float64
 7   accommodates                                  6936 non-null   int64  
 8   bedrooms                                      6936 non-null   float64
 9   beds                                          6936 non-null   f

### Guardar a CSV

In [53]:
#Convertir archivo filtrado a CSV
#filtroa.to_csv("filtroa_eua.csv")
#filtrob.to_csv("filtrob_eua.csv")
#filtroc.to_csv("filtroc_eua.csv")
#filtrod.to_csv("filtrod_eua.csv")
#filtroe.to_csv("filtroe_eua.csv")
#filtrof.to_csv("filtrof_eua.csv")
#filtrog.to_csv("filtrog_eua.csv")
#filtroh.to_csv("filtroh_eua.csv")
#filtroi.to_csv("filtroi_eua.csv")
#filtroj.to_csv("filtroj_eua.csv")

In [54]:
#descargar archivo filtrado en csv
from google.colab import files  
#files.download("filtroa_eua.csv")
#files.download("filtrob_eua.csv")
#files.download("filtroc_eua.csv")
#files.download("filtrod_eua.csv")
#files.download("filtroe_eua.csv")
#files.download("filtrof_eua.csv")
#files.download("filtrog_eua.csv")
#files.download("filtroh_eua.csv")
#files.download("filtroi_eua.csv")
#files.download("filtroj_eua.csv")

# ***Girona_España.csv***

## **ETAPA 1**

In [55]:
dfeua = pd.read_csv('Girona_España.csv', encoding = 'unicode_escape')
dfeua.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19655 entries, 0 to 19654
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            19655 non-null  float64
 1   listing_url                                   19655 non-null  object 
 2   scrape_id                                     19655 non-null  float64
 3   last_scraped                                  19655 non-null  object 
 4   source                                        19655 non-null  object 
 5   name                                          19655 non-null  object 
 6   description                                   19632 non-null  object 
 7   neighborhood_overview                         8342 non-null   object 
 8   picture_url                                   19655 non-null  object 
 9   host_id                                       19655 non-null 

In [56]:
#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=dfeua.isnull().sum().sum()
valores_nulos

174630

### Filtros

In [57]:
#Crear filtro numerico y no numerico
filtroeuanumerico = dfeua.select_dtypes(include=['float64', 'int64'])
#filtroeuanumerico.info()

In [58]:
filtroeua = dfeua.select_dtypes('object')
#filtroeuanonumerico.info()

### Nulos

In [59]:
#sustituir los nulos por el promedio de la columna
data1 = filtroeuanumerico.copy()
filtroeuanumerico = data1.fillna(data1.mean())


#sustituir los nulos por 'sin registro'
filtroeua = filtroeua.fillna('sin registro')


In [60]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

58965

In [61]:
#Checar si hay valores nulos por columna    NO NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeua.isnull().sum().sum()
valores_nulos

0

In [62]:
#NUMERICO
#Se eliminaron las columnas 'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated'
#ya que son numericas pero no tienen ningun registro y no aportan valor al análisis
filtroeuanumerico = filtroeuanumerico.drop(['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated'], axis=1)

#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

0

### Outliers **MÉTODO DE IQR (RANGO INTER QUARTILICO)**

In [63]:
#Método aplicando Cuartiles. Encuentro cuartiles 0.25 y 0.75
y=filtroeuanumerico

percentile25=y.quantile(0.25) #Q1
percentile75=y.quantile(0.75) #Q3
iqr= percentile75 - percentile25

Limite_Superior_iqr= percentile75 + 1.5*iqr
Limite_Inferior_iqr= percentile25 - 1.5*iqr
#print("Limite superior permitido", Limite_Superior_iqr)
#print("Limite inferior permitido", Limite_Inferior_iqr)

In [64]:
#Encontramos Ouliers
outliers_iqr= filtroeuanumerico[(y>Limite_Superior_iqr)|(y<Limite_Inferior_iqr)]
#outliers_iqr.head()

In [65]:
#Obtenemos datos limpios
data_clean_iqr1= filtroeuanumerico[(y<=Limite_Superior_iqr)&(y>=Limite_Inferior_iqr)]
#data_clean_iqr1.head()

In [66]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data_clean_iqr1.isnull().sum().sum()
valores_nulos

37109

In [67]:
#Reemplazamos valores atípicos (nulos) del dataframe con "mean"
#Realizamos una copia del dataframe
data5_iqr=filtroeuanumerico.copy()
data5_iqr=data5_iqr.fillna(round(filtroeuanumerico.mean(),1))
#data5_iqr

In [68]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data5_iqr.isnull().sum().sum()
valores_nulos

0

### Concatenar

In [69]:
concatenadoeua = pd.concat([data5_iqr, filtroeua], axis=1)
concatenadoeua.head()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,...,room_type,bathrooms_text,amenities,price,has_availability,calendar_last_scraped,first_review,last_review,license,instant_bookable
0,14551.0,2.023030e+13,57237,1.0,5.0,41.99650,2.83646,2,1.0,1.0,...,Private room,1 private bath,"[""Shared pool"", ""Refrigerator"", ""Long term sta...",$42.00,t,31/03/2023,08/10/2010,27/02/2023,HUTG-049542,f
1,1205757.0,2.023030e+13,6586355,1.0,1.0,41.79304,2.77412,2,1.0,1.0,...,Private room,1.5 shared baths,"[""Free parking on premises"", ""Refrigerator"", ""...",$47.00,t,31/03/2023,01/08/2013,26/06/2022,sin registro,f
2,1212820.0,2.023030e+13,6620645,2.0,3.0,42.11284,3.16670,4,1.0,2.0,...,Entire home/apt,1 bath,"[""Air conditioning"", ""Free parking on premises...",$74.00,t,01/04/2023,10/06/2013,30/12/2022,sin registro,f
3,751638.0,2.023030e+13,3680160,6.0,6.0,42.36958,2.98030,2,1.0,2.0,...,Private room,1 private bath,"[""Hair dryer"", ""Luggage dropoff allowed"", ""Fre...",$45.00,t,31/03/2023,23/08/2013,29/08/2022,PG000794,t
4,41342.0,2.023030e+13,180072,2.0,3.0,42.34655,3.20463,4,1.0,1.0,...,Entire home/apt,1 bath,"[""Free parking on premises"", ""Refrigerator"", ""...",$120.00,t,01/04/2023,14/07/2013,28/08/2021,HUTG-003688,f


### Guardar a CSV

In [70]:
#Convertir archivo filtrado a CSV
concatenadoeua.to_csv("ESP_limpio.csv")

In [71]:
#descargar archivo filtrado en csv
from google.colab import files  
files.download("ESP_limpio.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **ETAPA 2**

In [72]:
#filtroa
#se cambiaron los valores "sin registro" por "5%" para poder identificarlos ya que el dataset no tiene registros con ese valor
concatenadoeua["host_acceptance_rate"] = concatenadoeua["host_acceptance_rate"].replace("sin registro", "5%")
#se elimino el "%"
concatenadoeua['host_acceptance_rate'] = pd.to_numeric(concatenadoeua['host_acceptance_rate'].str.replace('%', ''))



In [73]:
#filtroe
#cambiar Half-bath, Shared half-bath, Private half-bath, sin registro
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.replace('Half-bath', '0.5').str.replace('Shared half-bath', '0.5').str.replace('Private half-bath', '0.5').str.replace('sin registro', '0')

#se extraen los numeros y se guardan como string en la misma columna
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.extract('(\d+\.?\d*)', expand=False)

#cambiar columna a float
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].astype(float)

In [74]:
#filtrof
#se elimino el "$" y ","
#se cambia la columna "price" a float para poder compararla
concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


<ipython-input-74-af53cb0f8d5c>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


In [75]:
filtroa_esp = concatenadoeua[concatenadoeua["host_acceptance_rate"] > 50]
filtrob_esp = concatenadoeua[concatenadoeua["host_is_superhost"] == "t"]
filtroc_esp = concatenadoeua[concatenadoeua["host_identity_verified"] == "f"]
filtrod_esp = concatenadoeua[concatenadoeua["property_type"].isin(["Private room", "Hotel room"])]
filtroe_esp = concatenadoeua[concatenadoeua["bathrooms_text"] > 1]
filtrof_esp = concatenadoeua.loc[(concatenadoeua['price'] > 10000) & (concatenadoeua['room_type'] == 'Entire home/apt')]
filtrog_esp = concatenadoeua[concatenadoeua["review_scores_cleanliness"] > 4.5]
filtroh_esp = concatenadoeua[concatenadoeua["review_scores_value"] > 4.9]
filtroi_esp = concatenadoeua[concatenadoeua["availability_365"] < 100]
filtroj_esp = concatenadoeua[concatenadoeua["host_response_time"] == "within an hour"]

### Guardar a CSV

In [76]:
#Convertir archivo filtrado a CSV
#filtroa.to_csv("filtroa_esp.csv")
#filtrob.to_csv("filtrob_esp.csv")
#filtroc.to_csv("filtroc_esp.csv")
#filtrod.to_csv("filtrod_esp.csv")
#filtroe.to_csv("filtroe_esp.csv")
#filtrof.to_csv("filtrof_esp.csv")
#filtrog.to_csv("filtrog_esp.csv")
#filtroh.to_csv("filtroh_esp.csv")
#filtroi.to_csv("filtroi_esp.csv")
#filtroj.to_csv("filtroj_esp.csv")

In [77]:
#descargar archivo filtrado en csv
from google.colab import files  
#files.download("filtroa_esp.csv")
#files.download("filtrob_esp.csv")
#files.download("filtroc_esp.csv")
#files.download("filtrod_esp.csv")
#files.download("filtroe_esp.csv")
#files.download("filtrof_esp.csv")
#files.download("filtrog_esp.csv")
#files.download("filtroh_esp.csv")
#files.download("filtroi_esp.csv")
#files.download("filtroj_esp.csv")

# ***DF_Mexico.csv***

## **ETAPA 1**

In [78]:
dfeua = pd.read_csv('DF_Mexico.csv')#, encoding = 'unicode_escape')
dfeua.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24224 entries, 0 to 24223
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            24224 non-null  int64  
 1   listing_url                                   24224 non-null  object 
 2   scrape_id                                     24224 non-null  int64  
 3   last_scraped                                  24224 non-null  object 
 4   source                                        24224 non-null  object 
 5   name                                          24223 non-null  object 
 6   description                                   23296 non-null  object 
 7   neighborhood_overview                         15145 non-null  object 
 8   picture_url                                   24224 non-null  object 
 9   host_id                                       24224 non-null 

In [79]:
#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=dfeua.isnull().sum().sum()
valores_nulos

189609

### Filtros

In [80]:
#Crear filtro numerico y no numerico
filtroeuanumerico = dfeua.select_dtypes(include=['float64', 'int64'])
#filtroeuanumerico.info()

In [81]:
filtroeua = dfeua.select_dtypes('object')
#filtroeuanonumerico.info()

### Nulos

In [82]:
#sustituir los nulos por el promedio de la columna
data1 = filtroeuanumerico.copy()
filtroeuanumerico = data1.fillna(data1.mean())


#sustituir los nulos por 'sin registro'
filtroeua = filtroeua.fillna('sin registro')


In [83]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

96896

In [84]:
#Checar si hay valores nulos por columna    NO NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=filtroeua.isnull().sum().sum()
valores_nulos

0

In [85]:
#NUMERICO
#Se eliminaron las columnas 'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated', 'license'
#ya que son numericas pero no tienen ningun registro y no aportan valor al análisis
filtroeuanumerico = filtroeuanumerico.drop(['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated','license'], axis=1)

#Checar si hay valores nulos por columna
pd.set_option('display.max_rows', None)
valores_nulos=filtroeuanumerico.isnull().sum().sum()
valores_nulos

0

### Outliers **MÉTODO DE IQR (RANGO INTER QUARTILICO)**

In [86]:
#Método aplicando Cuartiles. Encuentro cuartiles 0.25 y 0.75
y=filtroeuanumerico

percentile25=y.quantile(0.25) #Q1
percentile75=y.quantile(0.75) #Q3
iqr= percentile75 - percentile25

Limite_Superior_iqr= percentile75 + 1.5*iqr
Limite_Inferior_iqr= percentile25 - 1.5*iqr
#print("Limite superior permitido", Limite_Superior_iqr)
#print("Limite inferior permitido", Limite_Inferior_iqr)

In [87]:
#Encontramos Ouliers
outliers_iqr= filtroeuanumerico[(y>Limite_Superior_iqr)|(y<Limite_Inferior_iqr)]
#outliers_iqr.head()

In [88]:
#Obtenemos datos limpios
data_clean_iqr1= filtroeuanumerico[(y<=Limite_Superior_iqr)&(y>=Limite_Inferior_iqr)]
#data_clean_iqr1.head()

In [89]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data_clean_iqr1.isnull().sum().sum()
valores_nulos

54132

In [90]:
#Reemplazamos valores atípicos (nulos) del dataframe con "mean"
#Realizamos una copia del dataframe
data5_iqr=filtroeuanumerico.copy()
data5_iqr=data5_iqr.fillna(round(filtroeuanumerico.mean(),1))
#data5_iqr

In [91]:
#Checar si hay valores nulos por columna    NUMERICO
pd.set_option('display.max_rows', None)
valores_nulos=data5_iqr.isnull().sum().sum()
valores_nulos

0

### Concatenar

In [92]:
concatenadoeua = pd.concat([data5_iqr, filtroeua], axis=1)
concatenadoeua.head()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,...,property_type,room_type,bathrooms_text,amenities,price,has_availability,calendar_last_scraped,first_review,last_review,instant_bookable
0,2056638,20230329041210,10531228,3,7,19.35353,-99.16299,3,1.0,1.0,...,Entire loft,Entire home/apt,1.5 baths,"[""Wifi"", ""Hangers"", ""Fire extinguisher"", ""Sham...","$1,434.00",t,2023-03-30,2014-01-03,2018-06-24,f
1,2072354,20230329041210,16840050,3,3,19.35358,-99.16948,2,1.0,1.0,...,Entire guest suite,Entire home/apt,1 bath,"[""Free street parking"", ""Cleaning products"", ""...",$830.00,t,2023-03-30,2014-01-01,2022-11-04,f
2,696037,20230329041210,3531879,4,4,19.44180,-99.18402,6,3.0,3.0,...,Entire rental unit,Entire home/apt,2 baths,"[""Wifi \u2013 38 Mbps"", ""Central air condition...","$1,469.00",t,2023-03-29,2012-10-31,2022-12-21,f
3,35797,20230329041210,153786,1,1,19.38283,-99.27178,2,1.0,1.0,...,Entire villa,Entire home/apt,1 bath,"[""Wifi"", ""Washer"", ""BBQ grill"", ""Indoor firepl...","$3,658.00",t,2023-03-29,sin registro,sin registro,f
4,44616,20230329041210,196253,16,16,19.41162,-99.17794,14,5.0,8.0,...,Entire home,Entire home/apt,5.5 baths,"[""Free street parking"", ""Security cameras on p...","$18,000.00",t,2023-03-30,2011-11-09,2023-03-26,f


### Guardar a CSV

In [93]:
#Convertir archivo filtrado a CSV
concatenadoeua.to_csv("MEX_limpio.csv")

In [94]:
#descargar archivo filtrado en csv
from google.colab import files  
files.download("MEX_limpio.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **ETAPA 2**

In [95]:
#filtroa
#se cambiaron los valores "sin registro" por "5%" para poder identificarlos ya que el dataset no tiene registros con ese valor
concatenadoeua["host_acceptance_rate"] = concatenadoeua["host_acceptance_rate"].replace("sin registro", "5%")
#se elimino el "%"
concatenadoeua['host_acceptance_rate'] = pd.to_numeric(concatenadoeua['host_acceptance_rate'].str.replace('%', ''))



In [96]:
#filtroe
#cambiar Half-bath, Shared half-bath, Private half-bath, sin registro
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.replace('Half-bath', '0.5').str.replace('Shared half-bath', '0.5').str.replace('Private half-bath', '0.5').str.replace('sin registro', '0')

#se extraen los numeros y se guardan como string en la misma columna
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].str.extract('(\d+\.?\d*)', expand=False)

#cambiar columna a float
concatenadoeua["bathrooms_text"] = concatenadoeua["bathrooms_text"].astype(float)

In [97]:
#filtrof
#se elimino el "$" y ","
#se cambia la columna "price" a float para poder compararla
concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


<ipython-input-97-af53cb0f8d5c>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  concatenadoeua["price"] = concatenadoeua["price"].str.replace(',', '').str.replace('$', '').astype(float)


In [98]:
filtroa_mex = concatenadoeua[concatenadoeua["host_acceptance_rate"] > 50]
filtrob_mex = concatenadoeua[concatenadoeua["host_is_superhost"] == "t"]
filtroc_mex = concatenadoeua[concatenadoeua["host_identity_verified"] == "f"]
filtrod_mex = concatenadoeua[concatenadoeua["property_type"].isin(["Private room", "Hotel room"])]
filtroe_mex = concatenadoeua[concatenadoeua["bathrooms_text"] > 1]
filtrof_mex = concatenadoeua.loc[(concatenadoeua['price'] > 10000) & (concatenadoeua['room_type'] == 'Entire home/apt')]
filtrog_mex = concatenadoeua[concatenadoeua["review_scores_cleanliness"] > 4.5]
filtroh_mex = concatenadoeua[concatenadoeua["review_scores_value"] > 4.9]
filtroi_mex = concatenadoeua[concatenadoeua["availability_365"] < 100]
filtroj_mex = concatenadoeua[concatenadoeua["host_response_time"] == "within an hour"]

### Guardar a CSV

In [99]:
#Convertir archivo filtrado a CSV
#filtroa.to_csv("filtroa_mex.csv")
#filtrob.to_csv("filtrob_mex.csv")
#filtroc.to_csv("filtroc_mex.csv")
#filtrod.to_csv("filtrod_mex.csv")
#filtroe.to_csv("filtroe_mex.csv")
#filtrof.to_csv("filtrof_mex.csv")
#filtrog.to_csv("filtrog_mex.csv")
#filtroh.to_csv("filtroh_mex.csv")
#filtroi.to_csv("filtroi_mex.csv")
#filtroj.to_csv("filtroj_mex.csv")

In [100]:
#descargar archivo filtrado en csv
from google.colab import files  
#files.download("filtroa_mex.csv")
#files.download("filtrob_mex.csv")
#files.download("filtroc_mex.csv")
#files.download("filtrod_mex.csv")
#files.download("filtroe_mex.csv")
#files.download("filtrof_mex.csv")
#files.download("filtrog_mex.csv")
#files.download("filtroh_mex.csv")
#files.download("filtroi_mex.csv")
#files.download("filtroj_mex.csv")

# ***Insights filtros***

In [ ]:
#print(filtroj_eua.info(),filtroj_esp.info(),filtroj_mex.info())

Conclusión filtros

Para el filtro a primero que nada debieron cambiarse los valores en registro por un valor numérico, en este caso se cambiaron por el 5% ya que este valor no está en los registros por lo que de esta manera pueden ser identificados y posteriormente se eliminó el porcentaje presente en los registros. Después de eso se utilizó la función para filtrar las instancias cuyo valor es mayor a 50. En el data set de EUA sostuvieron 5679 instancias, en el de España 17,285 millones de Mexico 20,840. La diferencia de drástica de cantidades se deben parte del tamaño de las bases de datos, México siendo la más grande seguida por España y Estados Unidos.

El filtro b obtiene las instancias cuyo valor es correcto en la columna host_is_superhost en EUA se obtuvieron 2869 registros, en España 3056 registros y en México 8793 registros. 

El filtro c obtiene los valores falsos para la columna host_identity_verified, archivo de Estados Unidos 830 registros, en el de España 2261 registros y en el de Mexico 1923 registros.

El filtro d como a las instancias con valor de cuarto privado y de Cuarto de Hotel en la columna property_type. Es la tasa de Estados Unidos obtuvo cuatro registros, el de España siete y el de Mexico 43. 

Para el filtro e primero se tuvieron que cambiar los valores en texto a un número ya que estos no contenían ningún número por ejemplo medio baño. Después de televisión código para extraer los números de la columna y por último se cambió la columna a float. Esto para poder utilizar la función de filtrado para valores que son mayores a uno. En Estados Unidos sostuvieron 1844 registros, en España 9130 y en México 8904.

Para el filtro F  se eliminaron los signos de pesos y las comas de la columna y después se cambió a tipo float para después filtrar a valores mayores de 10,000 y que el tipo de cuarto sea igual a la casa completa. En Estados Unidos obtuvieron dos registros, en España 10 y en México 173.

El filtro G toma los valores mío es 4.5 en la columna review_scores_cleanliness en Estados Unidos sostuvieron 6127 registros, en España 14,690 y en México 21,445.

El filtro H similarmente al anterior, toma los valores mayores a 4.9 pero para la columna review_scores_value. En Estados Unidos obtuvieron 1376 registros, en España 2596 y en México 6484.

El filtro I toma los valores menores a 100 en la columna availability_365. En Estados Unidos se obtuvieron 2680 registros, en España 4981 y en México 5130.

El filtro J toma los valores de la columna host_response_time qué son iguales a menos de una hora. En Estados Unidos obtuvieron 4221 registros, en España 12,261 y por último en México 16,997.

Cabe mencionar que la discrepancia entre los tres archivos se puede ver en parte a la cantidad de registros en cada uno, será interesante comparar dichos datos con un porcentaje del total de los registros para cada una de las bases de datos en la siguiente etapa del entregable.